In [1]:
import torch
import torch.nn as nn
import timm
from tqdm import tqdm
import methods
import os
from astropy.io import fits
from astropy.wcs import WCS, utils
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np
import json
from io import BytesIO
import requests
from copy import deepcopy
from astropy.nddata import Cutout2D
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, random_split


In [2]:
oids, labels = methods.get_only_r_oids('akb.ztf.snad.space.json')
data = methods.FramesSequenceData(oids, labels)

In [3]:
np.random.seed(0)
torch.manual_seed(0)
x_train, _ = random_split(data, [0.01, 0.99])

In [4]:
bucket_boundaries = [200, 400, 600, 800]
sampler = methods.BySequenceLengthSampler(x_train, bucket_boundaries, 16, drop_last=False)


dataloader = DataLoader(x_train, batch_size=1, 
                        batch_sampler=sampler, 
                        num_workers=0, 
                        collate_fn=methods.collate,
                        drop_last=False, pin_memory=False)


In [3]:
model = methods.RBclassifier(hidden_size=128)

In [11]:
input_name = ["Images sequence"]
output_name = ["artefact or not"]

torch.onnx.export(model, data[100][0].float(), "rnn.onnx", input_names=input_name, output_names=output_name)

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/home/timofey/saimsu/astroenv/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4476: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [7]:
data[100][0].shape

torch.Size([312, 1, 28, 28])